## House Prediction with the help of Machine Learning Model
* Lasso Regression

#### Libraries we use

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

####    DataSet

In [2]:
df = pd.read_csv("train.csv")
print("Total Shape of our main dataset-->",df.shape)

Total Shape of our main dataset--> (1460, 81)


In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
#here we are creating our data for train and target
X = df.drop(columns=["Id","SalePrice"])  
y = df["SalePrice"]

In [5]:
print("Shape of you train and target data part-->",X.shape,y.shape,sep="\n")

Shape of you train and target data part-->
(1460, 79)
(1460,)


In [6]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


#### Droping Null Values

In [7]:
null_data = X.isnull().sum()/X.shape[0]*100
null_columns = null_data[null_data>20].keys()
print("Droping those columns have large amout of null values\n\n",null_columns)

Droping those columns have large amout of null values

 Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


In [8]:
X = X.drop(columns=null_columns)   #here we are dumping our null columns

In [9]:
print("Shape of our train Data-->",X.shape)

Shape of our train Data--> (1460, 74)


### Now we will split our data in object and number

##### Numerical Data

In [10]:
X_num = X.select_dtypes(include=["int64","float64"])
print("Columns present in our numerical Data Part\n\n",X_num.keys())

Columns present in our numerical Data Part

 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')


In [11]:
print("Number of null values present un our Numerial data",X_num.isnull().sum().sum())

Number of null values present un our Numerial data 348


In [12]:
num_nn = [var for var in X_num.columns if X_num[var].isnull().sum()>0]
print("Numerical data having null values\n\n",num_nn)

Numerical data having null values

 ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


##### String data`

In [13]:
X_char = X.select_dtypes(include=["object"])
print("Columns present in our Char Data Part\n\n",X_char.keys())

Columns present in our Char Data Part

 Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')


In [14]:
print("Number of null values present un our character data",X_char.isnull().sum().sum())

Number of null values present un our character data 520


In [15]:
char_nn = [var for var in X_char.columns if X_char[var].isnull().sum()>0]
print("Character data having null values\n\n",char_nn)

Character data having null values

 ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']


#### We have splitted our data now we will fill our non null values

In [16]:
#columns we have to use
num_nn = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
char_nn = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

In [17]:
#creating our piplne
mean_impute = Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])
mode_impute = Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])
#median_impute = Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])         -----> Giving Less accuracy

In [18]:
#creating our pipline merger
transform_1 = ColumnTransformer(transformers=
                              [("num_mean",mean_impute,num_nn),("char_mode",mode_impute,char_nn)])

In [19]:
#fitting our dataset
update_null = transform_1.fit_transform(X)
df_update_null = pd.DataFrame(update_null,columns=['LotFrontage', 'MasVnrArea', 'GarageYrBlt','MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'])

In [20]:
print("Total null values in our null_updation-->",df_update_null.isnull().sum().sum())

Total null values in our null_updation--> 0


In [21]:
X.update(df_update_null)

In [22]:
print("Total null values in our train data(X)-->",X.isnull().sum().sum())

Total null values in our train data(X)--> 0


In [23]:
print("Shape of train Data",X.shape,"\nShape of target Data",y.shape,sep="\n")

Shape of train Data
(1460, 74)

Shape of target Data
(1460,)


#### Label Encoding

In [24]:
lb = LabelEncoder()
k = np.array([lb.fit_transform(X[var]) for var in X_char.keys()])
k.resize((1460,38))
np = pd.DataFrame(k,columns=X_char.keys())
print("Shape of lb_df is-->",np.shape)

Shape of lb_df is--> (1460, 38)


In [25]:
X.update(np)
print("Shape of X is-->",X.shape)

Shape of X is--> (1460, 74)


## Creating our model

In [26]:
X_train,X_test, Y_train,Y_test = train_test_split(X,y,test_size=.3,random_state=42)

In [27]:
from sklearn.linear_model import Lasso

In [28]:
model=Lasso(alpha = 40)
model.fit(X_train,Y_train)
model.score(X_test,Y_test)

0.8184424304886406

#### Checking our error

In [29]:
y_pred = model.predict(X_test)

In [30]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [31]:
mse=mean_squared_error(Y_test,y_pred)
mse

1266925743.0585215

In [32]:
rmse=np.sqrt(mse)
rmse

35593.90036310325